# 29. 异常编码细节（Exception Coding Details）

本节关注工程细节：EAFP 风格、异常链（raise ... from ...）、资源管理（with）、以及如何让错误更可诊断。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 28 节：异常基础


## 知识点地图

- 1. EAFP vs LBYL：先做再处理失败 vs 先检查再做
- 2. 异常链：raise NewError(...) from e
- 3. with：资源释放的首选方式
- 4. 重新抛出：raise（不带参数）


## 自检清单（学完打勾）

- [ ] 理解 EAFP vs LBYL 两种风格
- [ ] 会用 raise ... from ... 保留原始原因
- [ ] 理解 __cause__/__context__ 的含义（第 30 节会深入）
- [ ] 优先用 with 管理资源（替代 finally 手动关闭）


## 知识点 1：EAFP vs LBYL：先做再处理失败 vs 先检查再做

Python 社区偏 EAFP：直接做，失败再捕获；但要捕获明确异常类型。


In [ ]:
d = {'a': 1}

# LBYL
if 'b' in d:
    print(d['b'])
else:
    print('missing')

# EAFP
try:
    print(d['b'])
except KeyError:
    print('missing')


## 知识点 2：异常链：raise NewError(...) from e

包装异常时保留原始原因，方便排障。


In [ ]:
def parse_positive_int(s):
    try:
        n = int(s)
    except ValueError as e:
        raise ValueError(f'not an int: {s!r}') from e
    if n <= 0:
        raise ValueError('must be positive')
    return n

try:
    parse_positive_int('x')
except Exception as e:
    print(type(e).__name__, e)
    print('cause:', type(e.__cause__).__name__)


## 知识点 3：with：资源释放的首选方式

with 能确保退出时释放资源（关闭文件/释放锁等），即使发生异常。


In [ ]:
from pathlib import Path
ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())
path = ART / 'tmp.txt'
path.write_text('hello', encoding='utf-8')

with path.open('r', encoding='utf-8') as f:
    print(f.read())


## 知识点 4：重新抛出：raise（不带参数）

在 except 内使用 raise 可原样重新抛出当前异常（保留 traceback）。


In [ ]:
try:
    int('x')
except ValueError:
    print('handled partially, re-raise')
    try:
        raise
    except ValueError as e:
        print('caught again:', type(e).__name__)


## 常见坑

- 捕获异常要具体：except Exception 会吞掉太多错误
- 包装异常时不要丢失原始异常（用 raise from）


## 综合小案例：实现友好的解析函数（保留 cause）

实现 parse_port(s)：必须是 1..65535 的整数；错误信息友好，并保留原始 ValueError。


In [ ]:
def parse_port(s):
    try:
        n = int(s)
    except ValueError as e:
        raise ValueError(f'port must be int, got {s!r}') from e
    if not (1 <= n <= 65535):
        raise ValueError('port out of range')
    return n

for x in ['8080', 'x', '70000']:
    try:
        print(parse_port(x))
    except Exception as e:
        print(type(e).__name__, e, 'cause=', type(e.__cause__).__name__ if e.__cause__ else None)


## 自测题（不写代码也能回答）

- EAFP 的核心思想是什么？
- raise ... from ... 解决什么问题？
- 为什么 with 比 finally 更推荐？


## 练习题（建议写代码）

- 实现 parse_positive_float(s)：错误时包装异常并保留 cause。
- 写一个函数，读取文件并返回行数，要求无论如何都关闭文件（用 with）。
